In [1]:
import configparser
import sys
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
sys.path.append('../')
sys.path.append('../Population/')
sys.path.append('../Engine/')

In [2]:
config = configparser.ConfigParser()
config.read('./config.txt')

['./config.txt']

In [3]:
from individual import individual
from population import population
from StateControl import decoder

In [4]:
pop = population(config=config)

In [5]:
ind = pop.get_population()
ind = ind[0]

In [6]:
decoder = decoder(config['individual setting'])

In [7]:
model = decoder.get_model(ind.get_dec())
model

[1] [7 7 4 1 1 7 2]
[7] [7 4 1 1 7 2 1]
[7] [4 1 1 7 2 1 6]
[4] [1 1 7 2 1 6 9]
[1] [1 7 2 1 6 9 7]


Sequential(
  (0): ConvolutionLayer(
    (conv): Conv2d(3, 210, kernel_size=(2, 2), stride=(1, 1))
    (bn): BatchNorm2d(210, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): ConvolutionLayer(
    (conv): Conv2d(210, 120, kernel_size=(2, 2), stride=(1, 1))
    (bn): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): ConvolutionLayer(
    (conv): Conv2d(120, 30, kernel_size=(2, 2), stride=(1, 1))
    (bn): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (3): ConvolutionLayer(
    (conv): Conv2d(30, 30, kernel_size=(2, 2), stride=(1, 1))
    (bn): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (4): ConvolutionLayer(
    (conv): Conv2d(30, 210, kernel_size=(3, 3), stride=(1, 1))
    (bn): BatchNorm2d(210, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [8]:
class linear(nn.Module):
    def __init__(self,inSize,channelSize):
        super(linear, self).__init__()
        self.inSize = inSize
        self.channelSize = channelSize
        self.fc1 = nn.Linear(inSize*inSize*channelSize, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = x.view(-1, self.inSize*self.inSize*self.channelSize)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
linearLayer = linear(decoder.fullConnectLayerSize,decoder.previousOutSize)

In [10]:
model = nn.Sequential(
    model,
    linearLayer
)

In [11]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='../Dataset/cafir10/', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='../Dataset/cafir10/', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [14]:
model.to(device)

Sequential(
  (0): Sequential(
    (0): ConvolutionLayer(
      (conv): Conv2d(3, 210, kernel_size=(2, 2), stride=(1, 1))
      (bn): BatchNorm2d(210, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ConvolutionLayer(
      (conv): Conv2d(210, 120, kernel_size=(2, 2), stride=(1, 1))
      (bn): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): ConvolutionLayer(
      (conv): Conv2d(120, 30, kernel_size=(2, 2), stride=(1, 1))
      (bn): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): ConvolutionLayer(
      (conv): Conv2d(30, 30, kernel_size=(2, 2), stride=(1, 1))
      (bn): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): ConvolutionLayer(
      (conv): Conv2d(30, 210, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(210, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): linear

In [15]:
import torch.nn.functional as F

In [ ]:
for epoch in range(30):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels =inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 0.184
[1,   400] loss: 0.149
[1,   600] loss: 0.133
[1,   800] loss: 0.128
[1,  1000] loss: 0.124
[1,  1200] loss: 0.119
[1,  1400] loss: 0.109
[2,   200] loss: 0.098
[2,   400] loss: 0.095
[2,   600] loss: 0.095
[2,   800] loss: 0.094
[2,  1000] loss: 0.094
[2,  1200] loss: 0.090
[2,  1400] loss: 0.089
[3,   200] loss: 0.076
[3,   400] loss: 0.075
[3,   600] loss: 0.076
[3,   800] loss: 0.074
[3,  1000] loss: 0.074
[3,  1200] loss: 0.076
[3,  1400] loss: 0.074
[4,   200] loss: 0.056
[4,   400] loss: 0.061
[4,   600] loss: 0.060
[4,   800] loss: 0.061
[4,  1000] loss: 0.060
[4,  1200] loss: 0.063
[4,  1400] loss: 0.059
[5,   200] loss: 0.044
[5,   400] loss: 0.046
[5,   600] loss: 0.043
[5,   800] loss: 0.045
[5,  1000] loss: 0.047
[5,  1200] loss: 0.049
[5,  1400] loss: 0.047
[6,   200] loss: 0.029
[6,   400] loss: 0.032
[6,   600] loss: 0.031
[6,   800] loss: 0.032
[6,  1000] loss: 0.031
[6,  1200] loss: 0.033
[6,  1400] loss: 0.035


In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        images, labels =images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
2257920/15210

In [ ]:
15210*32